<a href="https://colab.research.google.com/github/analyst-rhie/data-collection-information/blob/main/NLP_lecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# temsorflow 설치 확인 및 test
import tensorflow as tf 
a = tf.constant([10,10])

### 크롤링 계획 수립
* Global
  * 상업성/비상업성의 여부와 법적문제검토, 사용 가능 언어와 기술적 구현의 난이도를 고려
    * 크롤링 허용/불허용 확인 법 : robots.txt 웹 주소 뒤에 붙여두기
* 1. 크롤링 하려는 정보에 대한 정의 (어떤 데이터를 수집할 것인가?)
  * 어떠한 주식 데이터가 필요한가?
  * 수집해야하는 항목은?
  * 어떠한 형식으로 저장함? (excel, csv ...)

* 2. 해당 데이터를 수집하기에 가장 적합한 웹 페이지는 무엇인가?
  * 네이버 금융
  * Yahoo! Finance
  * 증권사 HTS

* 3. 어떤 기술적 방법으로 접근할 것인가?
  * 마우스 클릭 등의 동작이 필요한가?
  * 봇 제어 방지 시스템이 필요한가?
  * 효과적인 접근 방법은?


## 웹 글 추출법  1. BeautifulSoup

In [ ]:
# BeatifulSoup 확인
from bs4 import BeautifulSoup

In [ ]:
html_txt = '''
<html>
<body>
<h1 id="title> 자연어 처리 입문 강의<h1>
<p id="contents"> 코드 구현 을 중심 으로 진 행 합 니다. </p>
</body>
</html>
'''

In [ ]:
soup = BeautifulSoup(html_txt, 'lxml') #html 파싱방법보다 lxml 방법이 더 빠르다.
#lxml은 별도로 pip install 해야 함.

In [ ]:
# 뷰티플스푸가 html을 분석한 부분을 보여줌.
soup.html.body
soup.html.h1
soup.html.p

In [ ]:
# 한개의 요소를 찾을 때
soup = BeautifulSoup(html_txt, 'lxml') 
result = soup.find(element = name) # e.g. soup.find(id='h1')

# 여러 개의 요소를 찾을 때 
result=soup.find_all(element = name) #e.g.soup.find_all(id='h1')

* 웹 개발자 모드를 실행하기 위해 웹페이지에서 ctrl + shift + i 실행
  * 원하는 요소를 찾는 법 : 예를 들어 로그인/회원가입 웹 키를 두고 테스트 한다면 마우스 오른쪽 클릭 후 목록에서 검사를 클릭한다.

In [ ]:
import urllib.request as req

In [ ]:
#with문은 객체의 라이프사이클을 만들어 생성->출력->소멸 과정을 디자인 할 수 있다.
url = "https://learningspoons.com/"
with req.urlopen(url) as res : # 웹 개발자모드 실행
  soup = BeautifulSoup(res, 'lxml') # lxml 로 웹 사이트 구성요소 soup에 저장
  results= soup.select('li > a[href="https://learningspoons.com/account"]') # 원하는 조건을 선택해줌.
   # A > B 에서 A의 상위요소들 중 B의 하위요소를 선택하겠다는 뜻.(문자열이기 때문에 반드시 띄어쓰기 해줘야함)
   # 이때 a[ ] 구조는 a 내 href 로 구성된 하위요소들을 찾는데 사용한다.
  for idx, result in enumerate(results):
    print(idx, result)

0 <a class="nav-top-link nav-top-not-logged-in is-small" href="https://learningspoons.com/account">
<span>
    로그인     / 회원가입  </span>
</a>
1 <a class="account-link-mobile is-small" href="https://learningspoons.com/account" title="내 계정">
<i class="icon-user"></i> </a>
2 <a class="nav-top-link nav-top-not-logged-in" href="https://learningspoons.com/account">
<span class="header-account-title">
    로그인  </span>
</a>


* 실제 홈페이지에서 글자 가져와보기
  * href위치가 다 다르기 때문에 위에 처럼 가져올 수없음. 이때 공통적인 class를 찾아 가져온다.

In [ ]:
url = "https://online.learningspoons.com/"
with req.urlopen(url) as res :
  soup = BeautifulSoup(res, 'lxml')
  results = soup.select('h2[class="bb-course-title"] > a') # h2라는 조건들 중에서 class가 " ~"인 요소를 선택하고 a를 최종적으로 선택을 하겠다라는 의미
  for result in results :
    print(f'강의명:{result.string}')

강의명:처음 만나는 RPA : No코딩 업무 자동화로 진짜 일잘러 되기
강의명:고객의 숨은 니즈를 파악하는 유저 리서치 기반 UX 입문
강의명:[넷플 pick!] 리액티브 프로그래밍 with 람다(lambda)
강의명:완전 쉽게 익히는 R 데이터 분석
강의명:[취준생 주목!]코딩테스트를 위한 필수 알고리즘 정복&기출 문제 풀이 with 파이썬
강의명:박 회계사에게 배우는 반드시 오를 저평가 주식 발굴 및 가치평가
강의명:엑셀 파이낸셜 모델링 설계 및 구현 실무
강의명:하루 20분! 코알못을 위한 파이썬 기초 한달 완성
강의명:초보도 퍼포먼스 내는 디지털 마케팅 패키지 : 광고 집행, 데이터 분석, 자격증까지!
강의명:엑셀을 활용한 퀀트 투자 전략 입문
강의명:[취준생 주목!]코딩테스트를 위한 필수 알고리즘 정복&기출 문제 풀이 with 파이썬
강의명:대기업도 반한 템플릿 제작자의 PPT 기초 & 디자인
강의명:[100% 환급] 일잘러 마스터 패키지 : 엑셀/업무자동화/데이터분석/영상편집/회계까지
강의명:엑셀 VBA를 활용한 업무 자동화
강의명:수익모델을 만드는 에어비앤비 창업과 운영
강의명:박 회계사에게 배우는 반드시 오를 저평가 주식 발굴 및 가치평가
강의명:부동산 개발 및 사업수지 분석 기초
강의명:데이터 사이언티스트에게 배우는 파이썬 머신러닝 & 포트폴리오
강의명:고객의 숨은 니즈를 파악하는 유저 리서치 기반 UX 입문
강의명:어도비 디자인/영상편집 마스터 패키지
강의명:NLP 첫걸음! 자연어 처리 입문 완벽 가이드
강의명:펀드매니저에게 배우는 팩터 인베스팅
강의명:엑셀을 활용한 자산배분 퀀트 투자 전략
강의명:[엑셀 끝판왕] 함수, 데이터 분석, 자동화까지! 엑셀 마스터 패키지
강의명:회계 왕초보 탈출반
강의명:주식거래 시스템 구축 저자와 함께하는 북콘서트
강의명:엑셀 파이낸셜 모델링 설계 및 구현 실무
강의명:완전 쉽게 익히는 R 데이터 분석
강의명:파이썬 금융 프로그래밍 온라인 부트캠프
강의명:월 매출 5억의 운영자가 알려주는 스마트스토어 실전 마케